# Intersect catchment with GRACE 
# Finds the mean Aquifer Storage of each HRU in the model setup with rasterstats.

### Note
The rasterstats function `ZonalStatistics` automatically adds the calculated value to the shapefile used as input to the function. The workflow is thus:
1. Find the source catchment shapefile;
2. Copy the source catchment shapefile to the destintion location;
3. Run the zonal statistics algorithm on the copy.

In [113]:
# modules
import os
import sys
from pathlib import Path
from shutil import copyfile
from datetime import datetime
import geopandas as gpd
import rasterstats
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

#from qgis.core import QgsVectorLayer
#from qgis.core import QgsRasterLayer
#from qgis.analysis import QgsZonalStatistics

#### Control file handling

In [114]:
# Easy access to control file folder
controlFolder = Path('../../0_control_files')

In [115]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [116]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line and not line.startswith('#'):
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [117]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find location of shapefile and DEM

In [118]:
# Catchment shapefile path & name
catchment_path = read_from_control(controlFolder/controlFile,'catchment_shp_path')
catchment_name = read_from_control(controlFolder/controlFile,'catchment_shp_name')

In [119]:
# Specify default path if needed
if catchment_path == 'default':
    catchment_path = make_default_path('shapefiles/catchment') # outputs a Path()
else:
    catchment_path = Path(catchment_path) # make sure a user-specified path is a Path()

In [120]:
# DEM path & name
GRACE_path = read_from_control(controlFolder/controlFile,'observation_storage_GRACE_path')
GRACE_name = read_from_control(controlFolder/controlFile,'observation_storage_GRACE_name')

In [121]:
# Specify default path if needed
if GRACE_path == 'default':
    GRACE_path = make_default_path('observations/RS_Storage/GRACE/') # outputs a Path()
else:
    GRACE_path = Path(GRACE_path) # make sure a user-specified path is a Path()

GRACE_path

PosixPath('/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Storage/GRACE')

#### Find where the intersection needs to go

In [122]:
# Intersected shapefile path and name
intersect_path = read_from_control(controlFolder/controlFile,'intersect_GRACE_path')
intersect_name = read_from_control(controlFolder/controlFile,'intersect_GRACE_name')
print(intersect_name)
print(intersect_path)

catchment_with_GRACE.shp
default


In [123]:
# Specify default path if needed
if intersect_path == 'default':
    intersect_path = make_default_path('shapefiles/catchment_intersection/with_GRACE') # outputs a Path()
else:
    intersect_path = Path(intersect_path) # make sure a user-specified path is a Path()
intersect_path

PosixPath('/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/shapefiles/catchment_intersection/with_GRACE')

In [124]:
# Make the folder if it doesn't exist
intersect_path.mkdir(parents=True, exist_ok=True)

#### Copy the source catchment shapefile into the destination location

In [125]:
# Find the name without extension
catchment_base = catchment_name.replace('.shp','')

In [126]:
# Loop over directory contents and copy files that match the filename of the shape
for file in os.listdir(catchment_path):
    if catchment_base in file: # copy only the relevant files in case there are more than 1 .shp files
        
        # make the output file name
        _,ext = os.path.splitext(file)                    # extension of current file
        basefile,_ = os.path.splitext(intersect_name)     # name of the intersection file w/o extension
        newfile = basefile + ext                          # new name + old extension
        
        # copy
        copyfile(catchment_path/file, intersect_path/newfile);

## Get the SUMMA output file

In [127]:
simulation_path = read_from_control(controlFolder/controlFile,'experiment_output_summa')
simulation_name = read_from_control(controlFolder/controlFile,'experiment_id')

In [128]:
# Specify default path if needed
if simulation_path == 'default':
    simulation_path = make_default_path('simulations/' + simulation_name + '/SUMMA/' + simulation_name + '_day.nc')
    simulation_path = Path(simulation_path) # make sure a user-specified path is a Path()

simulation_path

PosixPath('/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/simulations/run_Yukon_Merit_1/SUMMA/run_Yukon_Merit_1_day.nc')

## Rasterstats analysis

In [129]:
ds = xr.open_dataset(simulation_path)

In [130]:
ds = ds.sel(time = slice('2002-04-01', '2016-12-31'))
ds

<xarray.Dataset> Size: 6GB
Dimensions:               (time: 5389, hru: 18225, gru: 18225)
Coordinates:
  * time                  (time) datetime64[ns] 43kB 2002-04-01 ... 2016-12-31
  * hru                   (hru) int64 146kB 81001460 81001703 ... 81036235
  * gru                   (gru) int64 146kB 81001460 81001703 ... 81036235
Data variables:
    scalarSWE             (time, hru) float64 786MB ...
    scalarAquiferStorage  (time, hru) float64 786MB ...
    scalarTotalSoilWat    (time, hru) float64 786MB ...
    scalarInfiltration    (time, hru) float64 786MB ...
    scalarTotalET         (time, hru) float64 786MB ...
    scalarTotalRunoff     (time, hru) float64 786MB ...
    scalarNetRadiation    (time, hru) float64 786MB ...
    hruId                 (hru) int64 146kB ...
    gruId                 (gru) int64 146kB ...

In [131]:
dsResampled = ds.resample(time='M').mean()
dsResampled

/Users/darrieythorsson/compHydro/data/CAMELS_spat/camels-spat-env/lib/python3.12/site-packages/xarray/core/groupby.py:532: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  index_grouper = pd.Grouper(


<xarray.Dataset> Size: 233MB
Dimensions:               (time: 177, hru: 18225, gru: 18225)
Coordinates:
  * hru                   (hru) int64 146kB 81001460 81001703 ... 81036235
  * gru                   (gru) int64 146kB 81001460 81001703 ... 81036235
  * time                  (time) datetime64[ns] 1kB 2002-04-30 ... 2016-12-31
Data variables:
    scalarSWE             (time, hru) float64 26MB 84.66 48.86 ... 37.76 63.76
    scalarAquiferStorage  (time, hru) float64 26MB 6.601e-06 ... 7.508e-05
    scalarTotalSoilWat    (time, hru) float64 26MB 1.461e+03 ... 1.157e+03
    scalarInfiltration    (time, hru) float64 26MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    scalarTotalET         (time, hru) float64 26MB 0.0 -5.974e-06 ... -7.535e-11
    scalarTotalRunoff     (time, hru) float64 26MB 3.561e-11 ... 4.644e-09
    scalarNetRadiation    (time, hru) float64 26MB 37.14 210.6 ... -29.77 -27.39
    hruId                 (time, hru) int64 26MB 81001460 81001703 ... 81036235
    gruId                 (time, gru) int64 26MB 81001460 81001703 ... 81036235

#### Spatial analysis

In [132]:
# Convert Path() to string for QGIS
catchment_file = str(intersect_path/intersect_name) # needs to be the coped file because output is automatically added to this
GRACE_file = str(GRACE_path/GRACE_name)

In [133]:
layer_polygon = catchment_file
layer_raster = GRACE_file

print(layer_raster)
print(layer_polygon)

/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Storage/GRACE/GRACE.tif
/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/shapefiles/catchment_intersection/with_GRACE/catchment_with_GRACE.shp


In [134]:
raster_file = rasterio.open(layer_raster)
raster_file.indexes

(1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159)

In [135]:
raster = np.array(raster_file.read(1)).astype(float)
raster

array([[ 9.3516407 ,  9.3516407 ,  9.3516407 , ..., -6.68717813,
        -6.68717813, -6.68717813],
       [ 9.3516407 ,  9.3516407 ,  9.3516407 , ..., -6.68717813,
        -6.68717813, -6.68717813],
       [ 9.3516407 ,  9.3516407 ,  9.3516407 , ..., -6.68717813,
        -6.68717813, -6.68717813],
       ...,
       [        nan,         nan,         nan, ..., 19.23863411,
        19.23863411, 19.23863411],
       [        nan,         nan,         nan, ..., 19.23863411,
        19.23863411, 19.23863411],
       [        nan,         nan,         nan, ..., 19.86211586,
        19.86211586, 19.86211586]])

In [136]:
raster_file = rasterio.open(layer_raster)
affine = raster_file.transform


Storage = []
for i in range(1,raster_file.count):#, dt in enumerate(dsResampled.time):

    array = np.array(raster_file.read(i+1)).astype(float)
    #array[array > 101.0] = np.nan
    zstats = rasterstats.zonal_stats(layer_polygon, array, affine=affine)
    zstats = pd.DataFrame(zstats)

    Storage.append(zstats['mean'])
    #shp['ndsi_mean'] = zstats['mean']

print(Storage)

KeyboardInterrupt: 

In [ ]:

dsResampled = dsResampled.isel(time = slice(1,len(Storage) + 1))

In [ ]:
dsResampled['Storage'] = (['time','hru'],Storage)

shp = gpd.read_file(layer_polygon)
correlation_map = xr.corr(dsResampled.Storage, ds.scalarAquiferStorage, dim ='time')
shp['GRACE_corr'] = abs(correlation_map) 


print(dsResampled)



<xarray.Dataset> Size: 231MB
Dimensions:               (time: 158, hru: 18225, gru: 18225)
Coordinates:
  * hru                   (hru) int64 146kB 81001460 81001703 ... 81036235
  * gru                   (gru) int64 146kB 81001460 81001703 ... 81036235
  * time                  (time) datetime64[ns] 1kB 2002-06-30 ... 2015-07-31
Data variables:
    scalarSWE             (time, hru) float64 23MB 54.9 17.07 18.96 ... 0.0 0.0
    scalarAquiferStorage  (time, hru) float64 23MB 8.391e-05 ... 0.0001647
    scalarTotalSoilWat    (time, hru) float64 23MB 1.543e+03 ... 1.306e+03
    scalarInfiltration    (time, hru) float64 23MB 1.249e-07 ... 2.128e-09
    scalarTotalET         (time, hru) float64 23MB -4.847e-06 ... -5.997e-05
    scalarTotalRunoff     (time, hru) float64 23MB 1.089e-07 ... 2.216e-08
    scalarNetRadiation    (time, hru) float64 23MB 116.1 278.9 ... 255.4 257.3
    hruId                 (time, hru) int64 23MB 81001460 81001703 ... 81036235
    gruId                 (time, gru

/Users/darrieythorsson/compHydro/data/CAMELS_spat/camels-spat-env/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/darrieythorsson/compHydro/data/CAMELS_spat/camels-spat-env/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
save_corr_file_path = read_from_control(controlFolder/controlFile,'observation_storage_correlation_path')
save_corr_file_name = read_from_control(controlFolder/controlFile,'observation_storage_correlation_name')

# Specify default path if needed
if save_corr_file_path == 'default':
    save_corr_file_path = make_default_path('evaluation/MOD16A2') # outputs a Path()
else:
    save_corr_file_path = Path(save_corr_file_path) # make sure a user-specified path is a Path()


shp.to_file(save_corr_file_path/Path(save_corr_file_name))

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [ ]:
# Set the log path and file name
logPath = intersect_path
log_suffix = '_catchment_dem_intersect_log.txt'

In [ ]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [ ]:
# Copy this script
thisFile = '1_find_HRU_elevation.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

FileNotFoundError: [Errno 2] No such file or directory: '1_find_HRU_elevation.ipynb'

In [ ]:
# Get current date and time
now = datetime.now()

In [ ]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Found mean HRU elevation from MERIT Hydro adjusted elevation DEM.']
    for txt in lines:
        file.write(txt)  